# Exercise 13.3: Dual Perceptron

In [ ]:
using Random
using Plots
using RDatasets
using COSMO
using JuMP
import LinearAlgebra
LA = LinearAlgebra;
include("optimization_library.jl")

In [ ]:
# Load the dataset with the iris flowers
iris = dataset("datasets", "iris")
first(iris, 5)

In [ ]:
# find the indices of the setosa and virginica species
setosa_indices = findall(x -> x .== "setosa", iris.Species)
virginica_indices = findall(x -> x .== "virginica", iris.Species);

In [ ]:
# combine the species indices
species_indices = vcat(virginica_indices, setosa_indices)

# create the data matrix containing data from of the features PetalWidth and PetalLength
data = Matrix(iris[species_indices,["PetalWidth", "PetalLength"]])

# create the vector containg the labels of each data point
labels = iris[species_indices,["Species"]]

# converting the string label into numeric label -1 or 1
binary_labels = [x.Species == "setosa" ? -1 : 1 for x in eachrow(labels)];

In [ ]:
# l: labels in -1 and +1 format
# x: input data
# max_iters: maximum number of iterations within which the algorithm should converge
function dual_perceptron(x, l, max_iters)
    α = zeros(Int64, size(x)[1])
    b = 0.0 
    # ================================================================================================
    #  Implement the dual Perceptron Algorithm given in the lecture.   
    # ================================================================================================
    
    return b, α  
end;

In [ ]:
b , α = dual_perceptron( data, binary_labels, 10000)
println("Total number of updates: ", sum(α))

In [ ]:
# evaluate weights vector of decision boundary
w = sum([α[j] * binary_labels[j] * data[j, :] for j in 1:size(data)[1]])

In [ ]:
# plot the datasets
scatter(iris.PetalWidth[setosa_indices], iris.PetalLength[setosa_indices], label="setosa", size = (350, 350))
scatter!(iris.PetalWidth[virginica_indices], iris.PetalLength[virginica_indices], label="virginica")

# plot the decision boundary
# run the dual_perceptron several times and observe the different decision boundaries!
x_axis = collect(0:3)
slope = - w[1] / w[2]
y_intercept = (-b / w[2]) 
a =  slope * x_axis .+ y_intercept
plot!(x_axis, a, label="decision boundary",  xlabel = "petal width/cm", ylabel= "petal length/cm", color= :red, legend=:bottomright)
ylims!((0,8))

# Exercise 13.4: Dual Support Vector Machine

In [ ]:
# define some kernel functions

function exponential_kernel(x, y; σ::Float64=1.0)
    # ================================================================================================
    #  Implement the exponential kernel
    # ================================================================================================
    return 0
end

# d: polynomial dregree
# c: constant offset
function polynomial_kernel(x, y; d=2, c=1.)
    # ================================================================================================
    #  Implement the polynomial kernel of degree d
    # ================================================================================================
    return 0
end


In [ ]:
# function solving the dual of the hard-margin SVM
# l: labels in -1 and +1 format
# x: input data
# k: Kernel function
function svm(x, l; k=dot)
    
    #build Matrix K from kernels and labels
    K = zeros(length(l), length(l))
    for i = 1:length(l), j=1:length(l)
        K[i,j] = l[i]*l[j]*k(x[i,:],x[j,:])
    end
    
    # ensure positive semi-definiteness of K that might be violated by Eigenvalues close to zero 
    # due to numerical imprecisions
    for i = 1:length(l)
        K[i,i] += 0.001
    end
      
    # ================================================================================================
    # define the objective and constraints of the optimization problem
    # solve the optimization problem
    # ================================================================================================
    
    μ = zeros(length(l))
    
    # return the values of the dual variable
    return μ
end;

In [ ]:
# Use output of svm() function to classify data points y
# l: labels in -1 and +1 format
# x: train data
# y: test data
# μ: dual variable
# k: Kernel function
function svm_classify(y, x, l, μ; k=dot)
    A = findall(μ .> 0.0001)
    result = 0
    # ================================================================================================
    #  Implement the computation of  <w | y> + b 
    # ================================================================================================
    
    return result
end

In [ ]:
# find the indices for the versicolor species
versicolor_indices = findall(x -> x .== "versicolor", iris.Species);

In [ ]:
# combine indices of virginica and versicolor species
species_indices = vcat(virginica_indices, versicolor_indices)

# create the data matrix containing data from virginica and versicolor species
# with measured features PetalWidth and PetalLength
data = Matrix(iris[species_indices,["PetalWidth", "PetalLength"]])

# create the vector containg the labels of each data point
labels = iris[species_indices,["Species"]]

# converting the string label into numeric label -1 or 1
binary_labels = [x.Species == "versicolor" ? -1 : 1 for x in eachrow(labels)];

In [ ]:
# Construct closures
# Try different hyperparameters!!!
function poly_kernel_closure(x, y)
    return polynomial_kernel(x,y,d=5)
end

function exp_kernel_closure(x, y)
    return exponential_kernel(x,y,σ = 0.5)
end

# choose a kernel function
kernel_function = exp_kernel_closure

# compute the result of the svm
μ = svm(data, binary_labels, k=kernel_function);

In [ ]:
# closure for svm_classify to simplify plotting
function svm_classify_closure(y1,y2)
    return svm_classify([y1,y2], data, binary_labels, μ,  k=kernel_function)
end

In [ ]:
ys = 2.0:0.04:8.0
xs = 0.0:0.04:3.0
contour(xs, ys, svm_classify_closure, fill=true, c=:bluesreds)#, colorbar=:none)# levels=[-1,0.5,0,0.5,1])
scatter!(iris.PetalWidth[versicolor_indices], iris.PetalLength[versicolor_indices], label="versicolor")
scatter!(iris.PetalWidth[virginica_indices], iris.PetalLength[virginica_indices], label="virginica")

ylims!((2,8))
xlims!((0,3))
plot!(size=(500,300), xlabel = "petal width/cm", ylabel= "petal length/cm",)